# Exploring sparse sliding windows

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py


## Convolution Inputs


In [ ]:
i = Tensor.fromUncompressed(["W"], [ 1, 2, 3, 4, 5, 0, 7, 0, 9, 0, 0, 12, 13, 0, 0, 0, 16, 17, 18])

displayTensor(i.setColor("blue"))

def printDistance(fiber, start_pos, window):
    if len(window) > 0:
        end_coord,_ = window[0]
        end_pos = fiber.getPosition(end_coord)
        print(f"Search distance: {end_pos-start_pos}")
    else:
        print(f"Search distance: unknown - window empty")




## Sliding window (by 3)

Optional start_pos keyword argument set to 0 to collect statistics on default behavior

In [ ]:

canvas = createCanvas(i)

i_w = i.getRoot()

S = 3
W = i_w.getShape()[0]
Q = W - S + 1

for q in range(0, Q):
    print(f"Search for q={q} from start position=0")
    
    window = i_w.getRange(q, S, start_pos=0)
    printDistance(i_w, 0, window)
    
    for w, i_val in window:
        pass
        
    win_coords = [(x,) for x in range(q, q+S)]
    addFrame(canvas, win_coords)

(count, distance) = i_w.getSavedPosStats()
print("")
print(f"Average search distance = {distance/count}")
    
displayCanvas(canvas)

## Sliding window (by 3) - with optimized search

Optional start_pos keyword argument set naively to reduce search for next window

In [ ]:

canvas = createCanvas(i)

i_w = i.getRoot()

S = 3
W = i_w.getShape()[0]
Q = W - S + 1

next_start_pos = 0

for q in range(0, Q):

    print(f"Search for q={q} from start position={next_start_pos}")

    window = i_w.getRange(q, S, start_pos=next_start_pos)
    printDistance(i_w, next_start_pos, window)

    # Next start position could be as much as window size (S) before
    # the position saved after the last search, but not less than the
    # last start position. 
    next_start_pos = max(next_start_pos, i_w.getSavedPos()-S)
    
    for w, i_val in window:
        pass
      
    win_coords = [(x,) for x in range(q, q+S)]
    addFrame(canvas, win_coords)

(count, distance) = i_w.getSavedPosStats()
print("")
print(f"Average search distance = {distance/count}")
    
displayCanvas(canvas)

## Sliding window (by 3) - with optimal search

Optional start_pos keyword argument set to reduce search for next window

In [ ]:

canvas = createCanvas(i)

i_w = i.getRoot()

S = 3
W = i_w.getShape()[0]
Q = W - S + 1

next_start_pos = 0

for q in range(0, Q):
    
    print(f"Search for q={q} from start position={next_start_pos}")

    window = i_w.getRange(q, S, start_pos=next_start_pos)
    printDistance(i_w, next_start_pos, window)
    
    start_pos_is_set = False
    
    for window_pos, (w, i_val) in enumerate(window):
        # Next start position is the first position with a 
        # coordinate greater than the last start coordinate (q).
        if not start_pos_is_set and w > q:
            start_pos_is_set = True
            next_start_pos = next_start_pos + window_pos
            
        pass
        

    # Maybe there is no element in the window with a coordinate
    # greater than the starting coordinate (q). So see if we should
    # keep looking at the same start position or bump it by one
    if not start_pos_is_set:
        cur_start_coord, _ = i_w[next_start_pos]
        if cur_start_coord < q + 1:
            next_start_pos += 1

    win_coords = [(x,) for x in range(q, q+S)]
    addFrame(canvas, win_coords)

(count, distance) = i_w.getSavedPosStats()
print("")
print(f"Average search distance = {distance/count}")
    
displayCanvas(canvas)

## Testing area

For running alternative algorithms